In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os
from PIL import Image
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, roc_auc_score, auc, average_precision_score
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt


In [ ]:
import tensorflow
import tensorflow.keras as keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess

from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import preprocess_input as vgg_preprocess

from tensorflow.keras.models import Model, Sequential,load_model

from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout, Input

In [ ]:
ResNet50().summary()

102973440/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

Dataset import and preprocessing

In [ ]:
with np.load('/content/gdrive/MyDrive/sp_train.npz') as data:
    X_train = data["dataset"]
    y_train = data["labels"]

with np.load('/content/gdrive/MyDrive/sp_valid.npz') as data:
    X_valid = data["dataset"]
    y_valid = data["labels"]

with np.load('/content/gdrive/MyDrive/sp_test.npz') as data:
    dataset = data["dataset"]
    labels = data["labels"]

In [ ]:
with np.load('/content/gdrive/MyDrive/sp_test.npz') as data:
    dataset = data["dataset"]
    labels = data["labels"]

In [ ]:
print("Training set samples: ", len(X_train))
print("Validation set samples: ", len(X_valid))
print("Test set samples: ", len(dataset))

Training set samples:  15390
Validation set samples:  3720
Test set samples:  400


In [ ]:
y_train = tensorflow.keras.utils.to_categorical(y_train, 2)
y_valid = tensorflow.keras.utils.to_categorical(y_valid, 2)

Model Training

In [ ]:
input_shape = (224, 224, 3)
input = dataset[:1]

base_model = ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False
output = base_model(input)
print(output.shape)

global_average_layer = keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(output)
print(feature_batch_average.shape)
prediction_layer = keras.layers.Dense(2,activation=keras.activations.softmax)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

inputs = keras.Input(shape=input_shape)
x = resnet_preprocess(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = Dropout(0.2)(x)
outputs = prediction_layer(x)
model = keras.Model(inputs, outputs)

94773248/94765736 [==============================] - 1s 0us/step
(1, 7, 7, 2048)
(1, 2048)
(1, 2)


In [ ]:
model.compile(optimizer=tensorflow.keras.optimizers.SGD(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy', 'AUC'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098  

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=16, epochs=1)

962/962 [==============================] - 67s 65ms/step - loss: 0.7927 - accuracy: 0.5180 - auc: 0.5256 - val_loss: 0.7125 - val_accuracy: 0.5296 - val_auc: 0.5472


In [ ]:
model.save('/content/gdrive/My Drive/models/sp_model_15_softmax2.h5')

Fine-tuning

In [ ]:
print(len(base_model.layers))
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 171

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

optimizer=keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())
fine_tune_epochs = 5
total_epochs =  15 + fine_tune_epochs

history_fine = model.fit(X_train,y_train,
                         validation_data=(X_valid, y_valid),
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         batch_size=16)

175
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 40

Model Evaluation

In [ ]:
model = tensorflow.keras.models.load_model('/content/gdrive/My Drive/models/sp_model_15_softmax2.h5')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098  

In [ ]:
preds = []
for i in range(len(dataset)):
  preds.append(np.argmax(tensorflow.nn.softmax(model.predict(dataset[i:i+1]), axis=-1)))

correct = []
for i in range(len(dataset)):
    correct.append(preds[i] == labels[i])

res = Counter(correct)
print(res)

Counter({True: 288, False: 112})


In [ ]:
res[1]/(res[0]+res[1])

0.72

In [ ]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(labels, preds)

In [ ]:
average_precision_score(labels, preds)

0.6609473684210527

In [ ]:
auc_keras = auc(fpr_keras, tpr_keras)
print(auc_keras)

0.72


In [ ]:
confusion_matrix()